PMI SVD
-------

Testing out the idea that useful word embeddings can be gleaned by performing the SVD on the matrix of pointwise mutual information between words and their co-occurrances. [Idea from here.](http://multithreaded.stitchfix.com/blog/2017/10/18/stop-using-word2vec/)

In [1]:
import re
import sys
import logging
import os
import itertools
import numpy as np
from sklearn.decomposition import TruncatedSVD as SVD
import collections

logging.basicConfig(format='%(asctime)s %(message)s', stream=sys.stdout, level="INFO")

Data
----

For this exercise I used MF DOOM song lyrics

In [2]:
data_dir = "../data/"

def process_file(directory, f):
    content = open(os.path.join(directory, f), "r").read()
    body = re.split('\n{2,}', content)
    return body[1].lower().split("\n")

def text_data(directory):
    all_files = reduce(lambda x,y: x+y, [files for sub, dirs, files in os.walk(directory)], [])
    text = [(f, process_file(directory, f)) for f in all_files]
    return dict(text)

file_text = text_data(data_dir)

In [3]:
def word_counts(text_dict):
    words = [word for file, lines in text_dict.iteritems() for line in lines for word in line.split()]
    return collections.Counter(words)

def skip_grams(line):
    return [tuple(sorted(list(t))) for t in itertools.combinations(line.split(), 2)]

def skip_gram_counts(text_dict):
    grams = [gram for file, lines in text_dict.iteritems() for line in lines for gram in skip_grams(line)]
    return collections.Counter(grams)


word_counter = word_counts(file_text)
gram_counter = skip_gram_counts(file_text)

word_index = dict([(word, index) for index, word in enumerate(word_counter.keys())])

In [4]:
"""
   is there a nice functional way to do this in python?
"""
word_count = len(word_index)
total_words = float(sum(word_counter.values()))
total_grams = float(sum(gram_counter.values()))

pmi_matrix = np.zeros((word_count, word_count))

for word1, row in word_index.iteritems():
    for word2, column in word_index.iteritems():
        tup = tuple(sorted([word1, word2]))
        
        gram_pct = gram_counter[tup]/total_grams
        word1_pct = word_counter[word1]/total_words
        word2_pct = word_counter[word2]/total_words
        
        pmi = 0 if gram_pct == 0 else np.log(gram_pct/(word1_pct*word2_pct))
        pmi_matrix[row, column] = pmi

In [5]:
pmi_svd = SVD(n_components=50).fit(pmi_matrix)

In [6]:
"""
   obviously this could be done more efficiently
"""
def similar_words(word, word_index, svd, to_get=10):
    if word not in word_index:
        return []
    else:
        index = word_index[word]
        vectors = svd.components_.T
        vector = vectors[index]
        
        index_to_word = dict([(i,w) for w,i in word_index.iteritems()])
        
        sims = [(index_to_word[index2], vector.dot(vector2)) for index2, vector2 in enumerate(vectors)]
    
        return sorted(sims, cmp=lambda x,y: cmp(x[1], y[1]), reverse=True)[:to_get]

In [7]:
similar_words("zygote", word_index, pmi_svd)

[('to', 0.0092093790167196501),
 ('just', 0.0078288557941133749),
 ('a', 0.0069691862547100166),
 ('hit', 0.0039355643370884881),
 ('all', 0.0038155953455745071),
 ('the', 0.003211757047555244),
 ("i'm", 0.0031908668861541467),
 ('time', 0.003039398076669351),
 ('happened', 0.002948875574251482),
 ('side', 0.0028487890991962597)]